# Model Registry

In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
# List all experiments
client.search_experiments()

[<Experiment: artifact_location='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/3', creation_time=1739805245793, experiment_id='3', last_update_time=1739805245793, lifecycle_stage='active', name='nyc-taxi-experiment-24', tags={}>,
 <Experiment: artifact_location='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/2', creation_time=1739801488499, experiment_id='2', last_update_time=1739801488499, lifecycle_stage='active', name='nyc-taxi-experiment-23', tags={}>,
 <Experiment: artifact_location='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/1', creation_time=1738849628527, experiment_id='1', last_update_time=1738849628527, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
# Create new experiment
client.create_experiment(name="my_cool_experiment")

'4'

In [13]:
from mlflow.entities import ViewType

# List runs
runs = client.search_runs(
    experiment_ids='3',
    filter_string="metrics.rmse < 12",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

runs

[<Run: data=<RunData: metrics={'rmse': 6.31451242969249}, params={'learning_rate': '0.2041',
  'max_depth': '17',
  'min_child_weight': '1.2402',
  'objective': 'reg:linear',
  'reg_alpha': '0.28567',
  'reg_lambda': '0.00426',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "c1ff55b974cb4607a8b32dacbd0b5e96", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2025-02-18 '
                              '13:32:27.427557", "model_uuid": '
                              '"35e7c95b491145d09e2a0e26a3245598", "flavors": '
                              '{"python_function": {"loader_module": '
                              '"mlflow.xgboost", "python_version": "3.8.10", '
                              '"data": "model.xgb", "env": {"conda": '
                              '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                              '"xgboost": {"xgb_version": "2.1.4", "data": '
             

In [14]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c1ff55b974cb4607a8b32dacbd0b5e96, rmse: 6.3145
run id: e6c8772ef4a442a6af5a6851f9ad8ddf, rmse: 6.3145
run id: 7cdfab6e658940578dee695f46fe7f3f, rmse: 6.3145
run id: d9da016e52ad473688a12b882b4980b9, rmse: 11.1673
run id: 6aefcd60503d49b6a7f696d038ec6425, rmse: 11.1673


In [12]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = "7cdfab6e658940578dee695f46fe7f3f"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1739890393831, current_stage='None', description=None, last_updated_timestamp=1739890393831, name='nyc-taxi-regressor', run_id='7cdfab6e658940578dee695f46fe7f3f', run_link=None, source='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/3/7cdfab6e658940578dee695f46fe7f3f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [17]:
# List all registered models
client.search_registered_models()

[<RegisteredModel: aliases={'production': 2, 'staging': 1}, creation_timestamp=1739888067544, description='The NYC Taxi Predictor for Trip Duration', last_updated_timestamp=1739890393831, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1739890393831, current_stage='None', description=None, last_updated_timestamp=1739890393831, name='nyc-taxi-regressor', run_id='7cdfab6e658940578dee695f46fe7f3f', run_link=None, source='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/3/7cdfab6e658940578dee695f46fe7f3f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>], name='nyc-taxi-regressor', tags={}>]

In [20]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version {version.version}, stage: {version.current_stage}")


version 1, stage: Production
version 2, stage: Staging
version 4, stage: None


/tmp/ipykernel_15359/3911191777.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [21]:
# Promote version 4 to staging
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Staging",
    archive_existing_versions=False
)

/tmp/ipykernel_15359/591136789.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1739890393831, current_stage='Staging', description=None, last_updated_timestamp=1739891019350, name='nyc-taxi-regressor', run_id='7cdfab6e658940578dee695f46fe7f3f', run_link=None, source='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/3/7cdfab6e658940578dee695f46fe7f3f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [23]:
from datetime import datetime

model_version = 4
new_stage = "Staging"
date = datetime.today().date()
# Update model

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1739890393831, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2025-02-18', last_updated_timestamp=1739891324915, name='nyc-taxi-regressor', run_id='7cdfab6e658940578dee695f46fe7f3f', run_link=None, source='/home/arief/Desktop/projects/mlops_zoomcamp/module_2/mlruns/3/7cdfab6e658940578dee695f46fe7f3f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

## A More End to End Process

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd

# Read dataframe
def read_dataframe(filename):
    
    df = pd.read_parquet(filename)    
    # Change column type
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

    # Get trip duration
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    # Change duration to minutes
    df['duration'] = df['duration'].apply(lambda d: d.total_seconds()/60)
    df = df.query('1 <= duration <=60')

    categorical = ['PULocationID', 'DOLocationID']
    # Change datatype to str so we can perform one hot encoding on it
    df[categorical] = df[categorical].astype(str)
    
    return df

# Preprocess
def preprocess(df, dv):

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    data_dicts = df[categorical + numerical].to_dict(orient='records')

    return dv.transform(data_dicts)

# Test model
def test_model(name, stage, X_test, y_test):

    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)

    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}